In [3]:
from sys import path
path.append(r"\\192.168.40.46\0. Sharing Folder\my-modules")

from Function_data_analyst import *
import sqlite3
import matplotlib.pyplot as plt


In [4]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",40)

In [11]:
db_local =cx_Oracle.connect("BIGDATA","BIGDATA#2020",cx_Oracle.makedsn("localhost",1521,"INFO"))
db_score =cx_Oracle.connect(user="score",password="Vmg102021",dsn="192.168.18.32:1521/score")
db_nice =cx_Oracle.connect(user="NICEDB",password="NiceVmg368",dsn="192.168.18.32:1521/NICEDB2")
db_bhxh_clean_crypt=sqlite3.connect(r"db_bhxh_clean_crypt.db")
db_bhxh=sqlite3.connect(r"BHXH.db")
db_1_to_many=sqlite3.connect("1_to_many.db")

# Create data sample

# Read data 

In [21]:
pd.read_sql_query("select * from sqlite_master",db_bhxh)

,type,name,tbl_name,rootpage,sql
0,table,S_BHXH_NEW,S_BHXH_NEW,2,"CREATE TABLE ""S_BHXH_NEW"" (\n""CMND_ENCRYPT"" TE..."
1,table,S_BHXH_NEW_DISTINCT,S_BHXH_NEW_DISTINCT,383602,CREATE TABLE S_BHXH_NEW_DISTINCT(\n CMND_ENCR...
2,table,S_BHXH_ONLINE_NEW,S_BHXH_ONLINE_NEW,687505,"CREATE TABLE ""S_BHXH_ONLINE_NEW"" (\n""CMND_ENCR..."
3,table,S_BHXH_ONLINE_NEW_DISTINCT,S_BHXH_ONLINE_NEW_DISTINCT,742965,CREATE TABLE S_BHXH_ONLINE_NEW_DISTINCT(\n CM...
4,table,BRANDNAME_SI_INDIVIDUAL,BRANDNAME_SI_INDIVIDUAL,798423,"CREATE TABLE ""BRANDNAME_SI_INDIVIDUAL"" (\n""USE..."
5,table,BRANDNAME_SI_INDIVIDUAL_DISTINCT,BRANDNAME_SI_INDIVIDUAL_DISTINCT,1503329,CREATE TABLE BRANDNAME_SI_INDIVIDUAL_DISTINCT(...
6,table,RESPONSE_BHXH_LOG,RESPONSE_BHXH_LOG,1953758,"CREATE TABLE ""RESPONSE_BHXH_LOG"" (\n""MA_SO_MXH..."
7,table,test,test,2938237,"CREATE TABLE ""test"" (\n""CMND_ENCRYPT"" TEXT,\n ..."
8,table,RESPONSE_BHXH_LOG_DISTINCT,RESPONSE_BHXH_LOG_DISTINCT,4170498,CREATE TABLE RESPONSE_BHXH_LOG_DISTINCT(\n MA...
9,table,BRANDNAME_EVN_BILL,BRANDNAME_EVN_BILL,6310420,"CREATE TABLE ""BRANDNAME_EVN_BILL"" (\n""MAKH"" TE..."


# Analyst data function

## Clean encrypt

In [6]:
def export_encrypt_key_to_db(df,name,conn,chunksize=500000):
    df=df.drop_duplicates()
    df.to_sql(name,conn_bhxh,index=False,if_exists="append",chunksize=chunksize)

def execute_db(conn, sql):
    try:
        c = conn.cursor()
        c.execute(sql)
    except Exception as e:
        print(e)
        
def create_distinct_table(conn,table_name):
    create_table_sql = """ CREATE TABLE IF NOT EXISTS {}_DISTINCT as 
    select distinct * from {};""".format(table_name,table_name)
    execute_db(conn, create_table_sql)

def check_nonunique_pair(db, table, key_field, compare_field):
    sql="""
    SELECT DISTINCT a.{key_field}, a.{compare_field} FROM {table} a
    INNER JOIN (SELECT {key_field} FROM (SELECT DISTINCT {key_field}, {compare_field} FROM {table}) 
    GROUP BY {key_field} HAVING COUNT({key_field})>1) b
    ON a.{key_field} = b.{key_field}
    ORDER by a.{key_field}
    """.format(key_field=key_field,compare_field=compare_field,table=table)
    return pd.read_sql_query( sql , db)

def pivot_analyst_field_data(ana,cols):
    total_row=pd.DataFrame(data=[ana.loc[0,"count(*)"]]*len(cols),columns=["TOTAL_ROW"],index=cols)
    count_non_nan_df=ana[["count({})".format(i) for i in cols]].set_axis(["COUNT_NOT_NAN"])
    count_non_nan_df.columns=cols
    count_distinct_df=ana[["count(distinct {})".format(i) for i in cols]].set_axis(["COUNT_DISTINCT"])
    count_distinct_df.columns=cols
    res=pd.concat([total_row,count_non_nan_df.T,count_distinct_df.T],axis=1)
    res["COUNT_NAN"]=res["TOTAL_ROW"]-res["COUNT_NOT_NAN"]
    res["COUNT_DUPLICATED"]=res["COUNT_NOT_NAN"]-res["COUNT_DISTINCT"]
    res["NOT_NAN_ON_TOTAL"]=(res["COUNT_NOT_NAN"]/res["TOTAL_ROW"]).round(4)
    res["NAN_ON_TOTAL"]=(res["COUNT_NAN"]/res["TOTAL_ROW"]).round(4)
    res["DISTINCT_ON_NOT_NAN"]=(res["COUNT_DISTINCT"]/res["COUNT_NOT_NAN"]).round(4)
    res["DUPLICATED_ON_NOT_NAN"]=(res["COUNT_DUPLICATED"]/res["COUNT_NOT_NAN"]).round(4)
    return res

def analyst_field_data(db,table):
    cols=read_data_from_db(db,table,n_records=0).columns.tolist()
    count_non_nan = ", ".join(["count({})".format(i) for i in ["*"] + cols])
    count_distinct = ", ".join(["count(distinct {})".format(i) for i in cols])
    sql="""select  {count_non_nan}, {count_distinct}  from {table}""".format(table=table,count_non_nan=count_non_nan,count_distinct=count_distinct)
    ana = pd.read_sql_query( sql , db)
    res = pivot_analyst_field_data(ana,cols)
    res.to_excel("analyst_field_data_{}.xlsx".format(table))
    return res

def clean_bhxh_func(chunk,db_map,clean_fields):
    if (db_map is None)==False:
        chunk=chunk.merge(db_map,how="left")
    chunk = DFCleaner(chunk,strip_cols = clean_fields)
    return chunk

def update_dict(row):
    try:
        if (row[0] is np.nan) or (row[0]["data"] is np.nan):
            row[0]={"data":[{}]}
        [x.update(row[1]) for x in row[0]["data"]]
    except:
        print(row)
        raise 

def clean_BHXH_LOG(df_sample,db_map=None,clean_fields=['MA_SO_MXH','SO_CMND',"IDCARD"]):
    df_sample=df_sample.drop_duplicates("RESPONSE_INFO").copy()
    df_sample= DFCleaner(df_sample)
    array_sample=np.array(df_sample.dropna().applymap(lambda x: eval(x.replace("null","np.nan")) if "{" in x  else np.nan,na_action="ignore").dropna())
    np.apply_along_axis(update_dict,1,array_sample)
    chunk=pd.DataFrame([item for sublist in array_sample for item in sublist[0]["data"]]).drop_duplicates()
    chunk.columns=[i.upper() for i in chunk.columns]
    for i in clean_fields:
        chunk[i]=chunk[i].astype(str).str.strip()
    return chunk 

def export_bhxh_todisk(db_con,table_name,col_name="*",db_map=None,clean_fields=["SOBHXH"],new_name_export=None,clean_func=clean_bhxh_func,
                       offset_rows=0,n_records=-1,chunksize=20000,reindex_col_tosql=None,to_db=db_bhxh):
    stt=1
    new_name_export = table_name if new_name_export is None else new_name_export
    for chunk in read_data_from_db(db_con=db_con,table_name=table_name,col_name=col_name,offset_rows=offset_rows,n_records=n_records,chunksize=chunksize):
        chunk=clean_func(chunk,db_map,clean_fields) if clean_func is not None else chunk
        chunk=chunk.reindex(reindex_col_tosql,axis=1) if (reindex_col_tosql is not None) else chunk
        chunk.to_sql(new_name_export,to_db,if_exists="append",index=False)
        print("done to rows :",offset_rows+stt*chunksize,end="\r")
        stt+=1

## Address

In [130]:
# read file map
add_map=pd.read_excel(r"\\192.168.40.116\Sharing Folder\final_address_map.xlsx")
add_map=add_map[['PROVINCE','PROVINCE_KEY']].drop_duplicates()
province=add_map.PROVINCE.tolist()

def get_province(x):
    convert=unidecode.unidecode(x.upper())
    res="NO_MAP"
    for i in province:
        if i in convert:
            res = i
            break
    return res

In [143]:
# dia chi
def count_address(db, table, col_name, add_map,chunksize=10000,n_records=50000):
    province=add_map.PROVINCE.tolist()
    re=pd.Series(dtype=int)
    address_chunk=read_data_from_db(db, table, col_name,n_records=n_records,chunksize=chunksize)
    stt=1
    for address in address_chunk:
        address = DFCleaner(address)
        re=re.add(address[col_name].dropna().map(get_province).value_counts(),fill_value=0)
        print("done row: ",chunksize*stt,end="\r")
        stt+=1
    res_groupby_key=re.reset_index().merge(add_map,how="left",left_on="index",right_on="PROVINCE").groupby("PROVINCE_KEY")[0].sum()
    return re,res_groupby_key

## Count digit

In [7]:
def count_digit(db,table,col,chunksize= 50000):
    count=0
    stt=1
    for chunk in read_data_from_db(db_con=db,table_name=table,col_name=col,offset_rows = 0,n_records = -1,chunksize= chunksize):
        count+=chunk[col].map(lambda x:str(x).isdigit()).sum()
        print(stt*chunksize,end="\r")
        stt+=1
    return count

In [23]:
execute_db(db_bhxh, "drop table if exists BRANDNAME_EVN_BILL")
execute_db(db_bhxh, "drop table if exists test")

# Analyst and report

## s_bhxh_new - db_score

### Clean data

In [8]:
# read key clean crypt
bhxh_new_clean_crypt=read_data_from_db(db_bhxh_clean_crypt,"S_BHXH_NEW_CLEAN",sql_query= "select distinct * from S_BHXH_NEW_CLEAN")

Bảng S_BHXH_NEW_CLEAN offset 0 dòng, all records


In [ ]:
export_bhxh_todisk(db_con=db_score,table_name="S_BHXH_NEW",col_name=["CMND_ENCRYPT","HOTEN_KD","NGAYSINH","GIOITINH","DIACHI","SOBHXH","NGAYCAP","NOICAP"]
                   ,db_map=bhxh_new_clean_crypt,clean_fields=["SOBHXH"],new_name_export=None,clean_func=clean_bhxh_func,
                       offset_rows=0,n_records=-1,chunksize=20000,reindex_col_tosql=None)

In [11]:
# tao bang distinct
create_distinct_table(db_bhxh,"S_BHXH_NEW")

In [18]:
read_data_from_db(db_bhxh,"S_BHXH_NEW_DISTINCT",n_records=0)

Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, 0 records


,CMND_ENCRYPT,HOTEN_KD,NGAYSINH,GIOITINH,DIACHI,SOBHXH,NGAYCAP,NOICAP,CMND_ENCRYPT_DECRYPT,CMND_ENCRYPT_CLEAN


### Overview

In [12]:
# print count total s_BHXH_new
print(read_data_from_db(db_bhxh,"S_BHXH_NEW",sql_query="select count(*) from S_BHXH_NEW").iloc[0,0])
# print count total s_BHXH_new_distinct
print(read_data_from_db(db_bhxh,"S_BHXH_NEW_DISTINCT",sql_query="select count(*) from S_BHXH_NEW_DISTINCT").iloc[0,0])

"""
Bảng S_BHXH_NEW offset 0 dòng, all records
9109378
Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, all records
7247859
"""

Bảng S_BHXH_NEW offset 0 dòng, all records
9109378
Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, all records
7247859


In [8]:

idclean_to_si_bhxhnew=check_nonunique_pair(db_bhxh,"S_BHXH_NEW_DISTINCT","CMND_ENCRYPT_CLEAN","SOBHXH")
idclean_to_si_bhxhnew.to_sql("idclean_to_si_bhxhnew",db_1_to_many,index=False, if_exists="replace")
si_to_idclean_bhxhnew=check_nonunique_pair(db_bhxh,"S_BHXH_NEW_DISTINCT","SOBHXH","CMND_ENCRYPT_CLEAN")
si_to_idclean_bhxhnew.to_sql("si_to_idclean_bhxhnew",db_1_to_many,index=False, if_exists="replace")
idclean_to_id_bhxhnew=check_nonunique_pair(db_bhxh,"S_BHXH_NEW_DISTINCT","CMND_ENCRYPT_CLEAN","CMND_ENCRYPT")
idclean_to_id_bhxhnew.to_sql("id_to_idclean_bhxhnew",db_1_to_many,index=False, if_exists="replace")
print(idclean_to_si_bhxhnew.apply(lambda x:x.nunique()))
print(si_to_idclean_bhxhnew.apply(lambda x:x.nunique()))
print(idclean_to_id_bhxhnew.apply(lambda x:x.nunique()))

CMND_ENCRYPT_CLEAN    352280
SOBHXH                758785
dtype: int64
SOBHXH                 72673
CMND_ENCRYPT_CLEAN    141400
dtype: int64
CMND_ENCRYPT_CLEAN    39113
CMND_ENCRYPT          83059
dtype: int64


In [24]:
masi=pd.read_sql_query("select * from si_to_id_bhxhlog where MA_SO_MXH is not null and SO_CMND is not null order by random() limit 30",db_1_to_many)["MA_SO_MXH"].tolist()

In [29]:
a=pd.read_sql_query("""select * from RESPONSE_BHXH_LOG_DISTINCT where MA_SO_MXH in ({})""".format(", ".join(['\'{}\''.format(i) for i in masi])),db_bhxh)

In [34]:
maid=pd.read_sql_query("select * from id_to_si_bhxhlog where MA_SO_MXH is not null and SO_CMND is not null order by random() limit 30",db_1_to_many)["SO_CMND"].tolist()
b=pd.read_sql_query("""select * from RESPONSE_BHXH_LOG_DISTINCT where SO_CMND in ({})""".format(", ".join(['\'{}\''.format(i) for i in maid])),db_bhxh)

In [35]:
a["Type"]="si_duplicated"
b["Type"]="id_duplicated"

In [36]:
a.append(b).to_excel("1_to_N_bhxh_log.xlsx")

In [12]:
pd.read_sql_query("""select * from sqlite_master where type = \'table\'""",db_1_to_many)

,type,name,tbl_name,rootpage,sql
0,table,idclean_to_si_bhxhnew,idclean_to_si_bhxhnew,2,"CREATE TABLE ""idclean_to_si_bhxhnew"" (\n""CMND_..."
1,table,is_to_idclean_bhxhnew,is_to_idclean_bhxhnew,8030,"CREATE TABLE ""is_to_idclean_bhxhnew"" (\n""SOBHX..."
2,table,id_to_idclean_bhxhnew,id_to_idclean_bhxhnew,9574,"CREATE TABLE ""id_to_idclean_bhxhnew"" (\n""CMND_..."
3,table,is_to_idclean_bhxhonlinenew,is_to_idclean_bhxhonlinenew,18770,"CREATE TABLE ""is_to_idclean_bhxhonlinenew"" (\n..."
4,table,si_to_phone_brandname,si_to_phone_brandname,21482,"CREATE TABLE ""si_to_phone_brandname"" (\n""MABHX..."
5,table,phone_to_si_brandname,phone_to_si_brandname,34914,"CREATE TABLE ""phone_to_si_brandname"" (\n""USER_..."
6,table,si_to_id_bhxhlog,si_to_id_bhxhlog,54879,"CREATE TABLE ""si_to_id_bhxhlog"" (\n""MA_SO_MXH""..."
7,table,id_to_si_bhxhlog,id_to_si_bhxhlog,54944,"CREATE TABLE ""id_to_si_bhxhlog"" (\n""SO_CMND"" T..."
8,table,idclean_to_si_bhxhonlinenew,idclean_to_si_bhxhonlinenew,10742,"CREATE TABLE ""idclean_to_si_bhxhonlinenew"" (\n..."
9,table,si_to_idclean_bhxhonlinenew,si_to_idclean_bhxhonlinenew,17865,"CREATE TABLE ""si_to_idclean_bhxhonlinenew"" (\n..."


In [20]:
ana=analyst_field_data(db_bhxh,"S_BHXH_NEW_DISTINCT")
ana

Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, 0 records


,count(*),count(CMND_ENCRYPT),count(HOTEN_KD),count(NGAYSINH),count(GIOITINH),count(DIACHI),count(SOBHXH),count(NGAYCAP),count(NOICAP),count(CMND_ENCRYPT_DECRYPT),count(CMND_ENCRYPT_CLEAN),count(distinct CMND_ENCRYPT),count(distinct HOTEN_KD),count(distinct NGAYSINH),count(distinct GIOITINH),count(distinct DIACHI),count(distinct SOBHXH),count(distinct NGAYCAP),count(distinct NOICAP),count(distinct CMND_ENCRYPT_DECRYPT),count(distinct CMND_ENCRYPT_CLEAN)
0,7247859,7247858,7247859,7232835,7234794,6979656,7247859,2726567,2832044,7247859,7247859,6052578,1055259,32160,3,3764317,6343810,15297,15752,6008632,6008632


In [50]:
res

,TOTAL_ROW,COUNT_NOT_NAN,COUNT_DISTINCT,COUNT_NAN,COUNT_DUPLICATED,NOT_NAN_ON_TOTAL,NAN_ON_TOTAL,DISTINCT_ON_NOT_NAN,DUPLICATED_ON_NOT_NAN
CMND_ENCRYPT,7247859,7247858,6052578,1,1195280,1.0000,0.0000,0.8351,0.1649
HOTEN_KD,7247859,7247859,1055259,0,6192600,1.0000,0.0000,0.1456,0.8544
NGAYSINH,7247859,7232835,32160,15024,7200675,0.9979,0.0021,0.0044,0.9956
GIOITINH,7247859,7234794,3,13065,7234791,0.9982,0.0018,0.0000,1.0000
DIACHI,7247859,6979656,3764317,268203,3215339,0.9630,0.0370,0.5393,0.4607
SOBHXH,7247859,7247859,6343810,0,904049,1.0000,0.0000,0.8753,0.1247
NGAYCAP,7247859,2726567,15297,4521292,2711270,0.3762,0.6238,0.0056,0.9944
NOICAP,7247859,2832044,15752,4415815,2816292,0.3907,0.6093,0.0056,0.9944
CMND_ENCRYPT_DECRYPT,7247859,7247859,6008632,0,1239227,1.0000,0.0000,0.8290,0.1710
CMND_ENCRYPT_CLEAN,7247859,7247859,6008632,0,1239227,1.0000,0.0000,0.8290,0.1710


### invalid value rate


In [55]:
# gioi tinh
pd.read_sql_query("select GIOITINH, count(GIOITINH) from S_BHXH_NEW_DISTINCT group by GIOITINH",db_bhxh)

,GIOITINH,count(GIOITINH)
0,None,0
1,0,3872495
2,1,3256144
3,2,106155


In [75]:
# ngay sinh
pd.read_sql_query("select length(NGAYSINH),count(length(NGAYSINH)) from S_BHXH_NEW_DISTINCT group by length(NGAYSINH)",db_bhxh)

,length(NGAYSINH),count(length(NGAYSINH))
0,NaN,0
1,4.0,12239
2,7.0,124
3,8.0,1
4,10.0,7220471


In [76]:
# len cmnd
pd.read_sql_query("select length(CMND_ENCRYPT_DECRYPT),count(length(CMND_ENCRYPT_DECRYPT)) from S_BHXH_NEW_DISTINCT group by length(CMND_ENCRYPT_DECRYPT)",db_bhxh)

,length(CMND_ENCRYPT_DECRYPT),count(length(CMND_ENCRYPT_DECRYPT))
0,0,10367
1,1,558
2,2,123
3,3,114
4,4,733
5,5,922
6,6,6068
7,7,2505
8,8,57202
9,9,7071983


In [77]:
# len bhxh
pd.read_sql_query("select length(SOBHXH),count(length(SOBHXH)) from S_BHXH_NEW_DISTINCT group by length(SOBHXH)",db_bhxh)

,length(SOBHXH),count(length(SOBHXH))
0,1,3
1,2,7
2,3,98
3,4,744
4,5,1804
5,6,13021
6,7,1854
7,8,160
8,9,3220
9,10,7226921


In [239]:
# count digit BHXH
a=count_digit(db_bhxh,"S_BHXH_NEW_DISTINCT","SOBHXH")
print("count digit bhxh",a)
# count digit CMND
a=count_digit(db_bhxh,"S_BHXH_NEW_DISTINCT","CMND_ENCRYPT_DECRYPT")
print("count digit cmnd",a)

Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit bhxh 5575371
Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit cmnd 7203543


### outlier 

### quantile

In [ ]:
# quantile before remove outlier

# quantile after remove outlier

### distribution

In [144]:
%%time
count_by_province,count_by_levelprovince=count_address(db_bhxh, "S_BHXH_NEW_DISTINCT", "DIACHI", add_map,chunksize=10000,    n_records=-1)

count_by_levelprovince

count_by_province.to_excel("count_by_province.xlsx")

## S_BHXH_ONLINE_NEW

### Download and clean

In [ ]:
bhxh_online_new_clean_crypt=read_data_from_db(db_bhxh_clean_crypt,"S_BHXH_ONLINE_NEW_CLEAN",sql_query= "select distinct * from S_BHXH_ONLINE_NEW_CLEAN")
export_bhxh_todisk(db_con=db_score,table_name="S_BHXH_ONLINE_NEW",col_name=["CMND_ENCRYPT","HOTEN_KD","HOTEN","NGAYSINH","GIOITINH","DIACHI","SOBHXH"],db_map=bhxh_online_new_clean_crypt)
# tao bang distinct
create_distinct_table(db_bhxh,"S_BHXH_ONLINE_NEW")

### Overview

In [164]:
# print count total s_BHXH_online_new
print(read_data_from_db(db_bhxh,"S_BHXH_ONLINE_NEW",sql_query="select count(*) from S_BHXH_ONLINE_NEW").iloc[0,0])
# print count total s_BHXH_online_new_distinct
print(read_data_from_db(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT",sql_query="select count(*) from S_BHXH_ONLINE_NEW_DISTINCT").iloc[0,0])

"""
Bảng S_BHXH_ONLINE_NEW offset 0 dòng, all records
1159758
Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, all records
1159724
"""

Bảng S_BHXH_ONLINE_NEW offset 0 dòng, all records
1159758
Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, all records
1159724


'\nBảng S_BHXH_ONLINE_NEW offset 0 dòng, all records\n9109378\nBảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, all records\n7247859\n'

In [62]:
ana2=analyst_field_data(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT")
ana2

Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, 0 records


,TOTAL_ROW,COUNT_NOT_NAN,COUNT_DISTINCT,COUNT_NAN,COUNT_DUPLICATED,NOT_NAN_ON_TOTAL,NAN_ON_TOTAL,DISTINCT_ON_NOT_NAN,DUPLICATED_ON_NOT_NAN
CMND_ENCRYPT,1159724,1159723,1159723,1,0,1.0,0.0,1.0000,0.0000
HOTEN_KD,1159724,1159724,227392,0,932332,1.0,0.0,0.1961,0.8039
HOTEN,1159724,1159724,285026,0,874698,1.0,0.0,0.2458,0.7542
NGAYSINH,1159724,1159724,27087,0,1132637,1.0,0.0,0.0234,0.9766
GIOITINH,1159724,1159724,2,0,1159722,1.0,0.0,0.0000,1.0000
DIACHI,1159724,1159724,3984,0,1155740,1.0,0.0,0.0034,0.9966
SOBHXH,1159724,1159724,1159724,0,0,1.0,0.0,1.0000,0.0000
CMND_ENCRYPT_DECRYPT,1159724,1159724,1157144,0,2580,1.0,0.0,0.9978,0.0022
CMND_ENCRYPT_CLEAN,1159724,1159724,1157144,0,2580,1.0,0.0,0.9978,0.0022


In [12]:
db_1_to_many=sqlite3.connect("1_to_many.db")
idclean_to_si_bhxhonlinenew=check_nonunique_pair(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT","CMND_ENCRYPT_CLEAN","SOBHXH")
idclean_to_si_bhxhonlinenew.to_sql("idclean_to_si_bhxhonlinenew",db_1_to_many,index=False, if_exists="replace")
si_to_idclean_bhxhonlinenew=check_nonunique_pair(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT","SOBHXH","CMND_ENCRYPT_CLEAN")
si_to_idclean_bhxhonlinenew.to_sql("si_to_idclean_bhxhonlinenew",db_1_to_many,index=False, if_exists="replace")
idclean_to_id_bhxhonlinenew=check_nonunique_pair(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT","CMND_ENCRYPT_CLEAN","CMND_ENCRYPT")
idclean_to_id_bhxhonlinenew.to_sql("idclean_to_id_bhxhonlinenew",db_1_to_many,index=False, if_exists="replace")
print(idclean_to_si_bhxhonlinenew.apply(lambda x:x.nunique()))
print(si_to_idclean_bhxhonlinenew.apply(lambda x:x.nunique()))
print(idclean_to_id_bhxhonlinenew.apply(lambda x:x.nunique()))

CMND_ENCRYPT_CLEAN       1
SOBHXH                2581
dtype: int64
SOBHXH                0
CMND_ENCRYPT_CLEAN    0
dtype: int64
CMND_ENCRYPT_CLEAN       1
CMND_ENCRYPT          2580
dtype: int64


### Invalid Value

In [168]:
# gioi tinh
pd.read_sql_query("select GIOITINH, count(GIOITINH) from S_BHXH_ONLINE_NEW_DISTINCT group by GIOITINH",db_bhxh)

,GIOITINH,count(GIOITINH)
0,0,544668
1,1,615056


In [169]:
# ngay sinh
pd.read_sql_query("select length(NGAYSINH),count(length(NGAYSINH)) from S_BHXH_ONLINE_NEW_DISTINCT group by length(NGAYSINH)",db_bhxh)

,length(NGAYSINH),count(length(NGAYSINH))
0,4,12239
1,7,124
2,8,1
3,10,1147360


In [170]:
# len cmnd
pd.read_sql_query("select length(CMND_ENCRYPT_DECRYPT),count(length(CMND_ENCRYPT_DECRYPT)) from S_BHXH_ONLINE_NEW_DISTINCT group by length(CMND_ENCRYPT_DECRYPT)",db_bhxh)

,length(CMND_ENCRYPT_DECRYPT),count(length(CMND_ENCRYPT_DECRYPT))
0,0,2584
1,1,2
2,2,5
3,3,5
4,4,2
5,6,2
6,7,24
7,8,218
8,9,1101567
9,10,5


In [171]:
# len bhxh
pd.read_sql_query("select length(SOBHXH),count(length(SOBHXH)) from S_BHXH_ONLINE_NEW_DISTINCT group by length(SOBHXH)",db_bhxh)

,length(SOBHXH),count(length(SOBHXH))
0,10,1159724


In [240]:
# count digit BHXH
a=count_digit(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT","SOBHXH")
print("count digit bhxh",a)
# count digit CMND
a=count_digit(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT","CMND_ENCRYPT_DECRYPT")
print("count digit cmnd",a)

Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit bhxh 1159724
Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit cmnd 1155990


### Distribution 

In [172]:
%%time
count_by_province,count_by_levelprovince=count_address(db_bhxh, "S_BHXH_ONLINE_NEW_DISTINCT", "DIACHI", add_map,chunksize=50000,    n_records=-1)
count_by_province.to_excel("count_by_province.xlsx")
count_by_levelprovince

Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, all records, chunksize 50000
Wall time: 1min 31s


PROVINCE_KEY
THANH PHO    1158955.0
TINH             587.0
Name: 0, dtype: float64

## BRANDNAME_SI_INDIVIDUAL

### Download and clean

In [ ]:
export_bhxh_todisk(db_con=db_nice,table_name="BRANDNAME_SI_INDIVIDUAL",clean_fields=["MABHXH","USER_ID"])
export_bhxh_todisk(db_con=db_nice,table_name="BRANDNAME_SI_INDIVIDUAL_B1",clean_fields=["MABHXH","USER_ID"],new_name_export="BRANDNAME_SI_INDIVIDUAL" )
# tao bang distinct
create_distinct_table(db_bhxh,"BRANDNAME_SI_INDIVIDUAL")

### Overview

In [173]:
# print count total BRANDNAME_SI_INDIVIDUAL
print(read_data_from_db(db_bhxh,"BRANDNAME_SI_INDIVIDUAL",sql_query="select count(*) from BRANDNAME_SI_INDIVIDUAL").iloc[0,0])
# print count total BRANDNAME_SI_INDIVIDUAL_distinct
print(read_data_from_db(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT",sql_query="select count(*) from BRANDNAME_SI_INDIVIDUAL_DISTINCT").iloc[0,0])

"""
Bảng BRANDNAME_SI_INDIVIDUAL offset 0 dòng, all records
50350860
Bảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, all records
32811496
"""

Bảng BRANDNAME_SI_INDIVIDUAL offset 0 dòng, all records
50350860
Bảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, all records
32811496


'\nBảng BRANDNAME_SI_INDIVIDUAL offset 0 dòng, all records\n9109378\nBảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, all records\n7247859\n'

In [63]:
ana3=analyst_field_data(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT")
ana3

Bảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, 0 records


,TOTAL_ROW,COUNT_NOT_NAN,COUNT_DISTINCT,COUNT_NAN,COUNT_DUPLICATED,NOT_NAN_ON_TOTAL,NAN_ON_TOTAL,DISTINCT_ON_NOT_NAN,DUPLICATED_ON_NOT_NAN
USER_ID,32811496,32811496,14420650,0,18390846,1.0000,0.0000,0.4395,0.5605
REPORT_DATE,32811496,32811495,517,1,32810978,1.0000,0.0000,0.0000,1.0000
BRANDNAME,32811496,32811495,1,1,32811494,1.0000,0.0000,0.0000,1.0000
MONTH,32811496,32811495,24,1,32811471,1.0000,0.0000,0.0000,1.0000
TEMPLATE_ID,32811496,32811495,37,1,32811458,1.0000,0.0000,0.0000,1.0000
TEN,32811496,7995181,754909,24816315,7240272,0.2437,0.7563,0.0944,0.9056
MABHXH,32811496,32811496,15061510,0,17749986,1.0000,0.0000,0.4590,0.5410
SOTIEN,32811496,6,6,32811490,0,0.0000,1.0000,1.0000,0.0000


In [10]:
db_1_to_many=sqlite3.connect("1_to_many.db")
si_to_phone_brandname=check_nonunique_pair(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","MABHXH","USER_ID")
si_to_phone_brandname.to_sql("si_to_phone_brandname",db_1_to_many,index=False, if_exists="replace")
phone_to_si_brandname=check_nonunique_pair(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","USER_ID","MABHXH")
phone_to_si_brandname.to_sql("phone_to_si_brandname",db_1_to_many,index=False, if_exists="replace")
print(si_to_phone_brandname.apply(lambda x:x.nunique()))
print(phone_to_si_brandname.apply(lambda x:x.nunique()))

MABHXH      886798
USER_ID    1704070
dtype: int64
USER_ID    1114760
MABHXH     2599083
dtype: int64


In [176]:
check_nonunique_pair(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","MABHXH","USER_ID").apply(lambda x:x.nunique())

MABHXH      886798
USER_ID    1704070
dtype: int64

In [177]:
check_nonunique_pair(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","USER_ID","MABHXH").apply(lambda x:x.nunique())

USER_ID    1114760
MABHXH     2599083
dtype: int64

### Invalid Value

In [178]:
# len cmnd
pd.read_sql_query("select length(USER_ID),count(length(USER_ID)) from BRANDNAME_SI_INDIVIDUAL_DISTINCT group by length(USER_ID)",db_bhxh)

,length(USER_ID),count(length(USER_ID))
0,3,1
1,9,248479
2,11,32363017
3,13,199999


In [179]:
# len bhxh
pd.read_sql_query("select length(MABHXH),count(length(MABHXH)) from BRANDNAME_SI_INDIVIDUAL_DISTINCT group by length(MABHXH)",db_bhxh)

,length(MABHXH),count(length(MABHXH))
0,1,7
1,3,1004
2,5,1
3,9,1
4,10,32810483


In [237]:
# count digit BHXH
a=count_digit(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","MABHXH")
print("count digit bhxh",a)
# count digit CMND
a=count_digit(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","USER_ID")
print("count digit cmnd",a)

Bảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, all records, chunksize 50000


32810354

In [241]:
# count digit CMND
a=count_digit(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT","USER_ID")
print("count digit cmnd",a)

Bảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit cmnd 32611496


### Distribution 

## RESPONSE_BHXH_LOG - db_score

### Download and clean

In [ ]:
export_bhxh_todisk(db_con=db_score,table_name="RESPONSE_BHXH_LOG",col_name=['RESPONSE_INFO','REQUEST_INFO'],
                   db_map=None,clean_fields=['MA_SO_MXH','SO_CMND',"IDCARD"],clean_func=clean_BHXH_LOG,reindex_col_tosql=x,
                  n_records=-1,chunksize=20000,offset_rows=2400000)
# tao bang distinct
create_distinct_table(db_bhxh,"RESPONSE_BHXH_LOG")

### Overview

In [174]:
# print count total RESPONSE_BHXH_LOG
print(read_data_from_db(db_bhxh,"RESPONSE_BHXH_LOG",sql_query="select count(*) from RESPONSE_BHXH_LOG").iloc[0,0])
# print count total RESPONSE_BHXH_LOG_distinct
print(read_data_from_db(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT",sql_query="select count(*) from RESPONSE_BHXH_LOG_DISTINCT").iloc[0,0])

"""
Bảng RESPONSE_BHXH_LOG offset 0 dòng, all records
9109378
Bảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, all records
7247859
"""

Bảng RESPONSE_BHXH_LOG offset 0 dòng, all records
23583738
Bảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, all records
22733562


'\nBảng RESPONSE_BHXH_LOG offset 0 dòng, all records\n9109378\nBảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, all records\n7247859\n'

In [175]:
ana4=analyst_field_data(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT")
ana4

Bảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, 0 records


,TOTAL_ROW,COUNT_NOT_NAN,COUNT_DISTINCT,COUNT_NAN,COUNT_DUPLICATED,NOT_NAN_ON_TOTAL,NAN_ON_TOTAL,DISTINCT_ON_NOT_NAN,DUPLICATED_ON_NOT_NAN
MA_SO_MXH,22733562,22733562,1660151,0,21073411,1.0000,0.0000,0.0730,0.9270
NOI_CAP_CMND,22733562,11574429,8148,11159133,11566281,0.5091,0.4909,0.0007,0.9993
HO_TEN,22733562,22733515,502575,47,22230940,1.0000,0.0000,0.0221,0.9779
MUC_LU,22733562,22733521,415462,41,22318059,1.0000,0.0000,0.0183,0.9817
DIA_CHI_HK,22733562,16133440,375640,6600122,15757800,0.7097,0.2903,0.0233,0.9767
TU_THANG,22733562,22733521,528,41,22732993,1.0000,0.0000,0.0000,1.0000
DEN_THANG,22733562,22733521,456,41,22733065,1.0000,0.0000,0.0000,1.0000
NGAY_CAP_CMND,22733562,9034034,12182,13699528,9021852,0.3974,0.6026,0.0013,0.9987
SO_CMND,22733562,22733562,1507760,0,21225802,1.0000,0.0000,0.0663,0.9337
VT_CTAC,22733562,14540515,702397,8193047,13838118,0.6396,0.3604,0.0483,0.9517


In [242]:
check_nonunique_pair(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT","MA_SO_MXH","SO_CMND").apply(lambda x:x.nunique())

MA_SO_MXH    4632
SO_CMND      9055
dtype: int64

In [243]:
check_nonunique_pair(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT","SO_CMND","MA_SO_MXH").apply(lambda x:x.nunique())

SO_CMND      102807
MA_SO_MXH    259802
dtype: int64

In [11]:
db_1_to_many=sqlite3.connect("1_to_many.db")
si_to_id_bhxhlog=check_nonunique_pair(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT","MA_SO_MXH","SO_CMND")
si_to_id_bhxhlog.to_sql("si_to_id_bhxhlog",db_1_to_many,index=False, if_exists="replace")
id_to_si_bhxhlog=check_nonunique_pair(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT","SO_CMND","MA_SO_MXH")
id_to_si_bhxhlog.to_sql("id_to_si_bhxhlog",db_1_to_many,index=False, if_exists="replace")
print(si_to_id_bhxhlog.apply(lambda x:x.nunique()))
print(id_to_si_bhxhlog.apply(lambda x:x.nunique()))

MA_SO_MXH    4632
SO_CMND      9055
dtype: int64
SO_CMND      102807
MA_SO_MXH    259802
dtype: int64


### Invalid Value

In [244]:
# gioi tinh
pd.read_sql_query("select GIOI_TINH, count(GIOI_TINH) from RESPONSE_BHXH_LOG_DISTINCT group by GIOI_TINH",db_bhxh)

,GIOI_TINH,count(GIOI_TINH)
0,NaN,0
1,0.0,11037328
2,1.0,11329044


In [245]:
# ngay sinh
pd.read_sql_query("select length(NGAY_SINH),count(length(NGAY_SINH)) from RESPONSE_BHXH_LOG_DISTINCT group by length(NGAY_SINH)",db_bhxh)

,length(NGAY_SINH),count(length(NGAY_SINH))
0,NaN,0
1,20.0,2
2,22.0,131
3,28.0,22733377


In [246]:
# len cmnd
pd.read_sql_query("select length(SO_CMND),count(length(SO_CMND)) from RESPONSE_BHXH_LOG_DISTINCT group by length(SO_CMND)",db_bhxh)

,length(SO_CMND),count(length(SO_CMND))
0,0,138
1,1,22166
2,2,93103
3,3,54676
4,4,1775
5,5,7
6,6,131
7,7,26
8,8,7518
9,9,19991493


In [247]:
# len bhxh
pd.read_sql_query("select length(MA_SO_MXH),count(length(MA_SO_MXH)) from RESPONSE_BHXH_LOG_DISTINCT group by length(MA_SO_MXH)",db_bhxh)

,length(MA_SO_MXH),count(length(MA_SO_MXH))
0,0,594
1,3,49
2,6,26
3,8,4
4,9,56
5,10,22732833


In [249]:
# count digit BHXH
a=count_digit(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT","MA_SO_MXH")
print("count digit bhxh",a)
# count digit CMND
a=count_digit(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT","SO_CMND")
print("count digit cmnd",a)

Bảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit bhxh 22652554
Bảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, all records, chunksize 50000
count digit cmnd 22697964


### Distribution 

# Coverage Map 

In [6]:
pd.read_sql_query("select * from sqlite_master",db_bhxh)

,type,name,tbl_name,rootpage,sql
0,table,S_BHXH_NEW,S_BHXH_NEW,2,"CREATE TABLE ""S_BHXH_NEW"" (\n""CMND_ENCRYPT"" TE..."
1,table,S_BHXH_NEW_DISTINCT,S_BHXH_NEW_DISTINCT,383602,CREATE TABLE S_BHXH_NEW_DISTINCT(\n CMND_ENCR...
2,table,S_BHXH_ONLINE_NEW,S_BHXH_ONLINE_NEW,687505,"CREATE TABLE ""S_BHXH_ONLINE_NEW"" (\n""CMND_ENCR..."
3,table,S_BHXH_ONLINE_NEW_DISTINCT,S_BHXH_ONLINE_NEW_DISTINCT,742965,CREATE TABLE S_BHXH_ONLINE_NEW_DISTINCT(\n CM...
4,table,BRANDNAME_SI_INDIVIDUAL,BRANDNAME_SI_INDIVIDUAL,798423,"CREATE TABLE ""BRANDNAME_SI_INDIVIDUAL"" (\n""USE..."
5,table,BRANDNAME_SI_INDIVIDUAL_DISTINCT,BRANDNAME_SI_INDIVIDUAL_DISTINCT,1503329,CREATE TABLE BRANDNAME_SI_INDIVIDUAL_DISTINCT(...
6,table,RESPONSE_BHXH_LOG,RESPONSE_BHXH_LOG,1953758,"CREATE TABLE ""RESPONSE_BHXH_LOG"" (\n""MA_SO_MXH..."
7,table,test,test,2938237,"CREATE TABLE ""test"" (\n""CMND_ENCRYPT"" TEXT,\n ..."
8,table,RESPONSE_BHXH_LOG_DISTINCT,RESPONSE_BHXH_LOG_DISTINCT,4170498,CREATE TABLE RESPONSE_BHXH_LOG_DISTINCT(\n MA...


In [14]:
read_data_from_db(db_bhxh,"S_BHXH_NEW_DISTINCT",n_records=0)

Bảng S_BHXH_NEW_DISTINCT offset 0 dòng, 0 records


,CMND_ENCRYPT,HOTEN_KD,NGAYSINH,GIOITINH,DIACHI,SOBHXH,NGAYCAP,NOICAP,CMND_ENCRYPT_DECRYPT,CMND_ENCRYPT_CLEAN


In [15]:
read_data_from_db(db_bhxh,"S_BHXH_ONLINE_NEW_DISTINCT",n_records=0)

Bảng S_BHXH_ONLINE_NEW_DISTINCT offset 0 dòng, 0 records


,CMND_ENCRYPT,HOTEN_KD,HOTEN,NGAYSINH,GIOITINH,DIACHI,SOBHXH,CMND_ENCRYPT_DECRYPT,CMND_ENCRYPT_CLEAN


In [13]:
read_data_from_db(db_bhxh,"BRANDNAME_SI_INDIVIDUAL_DISTINCT",n_records=0)

Bảng BRANDNAME_SI_INDIVIDUAL_DISTINCT offset 0 dòng, 0 records


,USER_ID,REPORT_DATE,BRANDNAME,MONTH,TEMPLATE_ID,TEN,MABHXH,SOTIEN


In [12]:
read_data_from_db(db_bhxh,"RESPONSE_BHXH_LOG_DISTINCT",n_records=0)

Bảng RESPONSE_BHXH_LOG_DISTINCT offset 0 dòng, 0 records


,MA_SO_MXH,NOI_CAP_CMND,HO_TEN,MUC_LU,DIA_CHI_HK,TU_THANG,DEN_THANG,NGAY_CAP_CMND,SO_CMND,VT_CTAC,NGAY_SINH,TEN_CONGTY,DIA_CHI_CTAC,GIOI_TINH,USER,IDCARD,IDFACEBOOK,NAME,DIABAN_CTAC,TEN_NGHIEPVU,ID,MUC_LUTT,HE_SO,MSKD,MANV,MSO


In [ ]:
 # new + online new

In [16]:
pd.read_sql_query("""select count(*) from (select distinct CMND_ENCRYPT_DECRYPT from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct CMND_ENCRYPT_DECRYPT from  S_BHXH_ONLINE_NEW_DISTINCT) b 
where a.CMND_ENCRYPT_DECRYPT=b.CMND_ENCRYPT_DECRYPT""",db_bhxh)

,count(*)
0,1152270


In [17]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct SOBHXH from  S_BHXH_ONLINE_NEW_DISTINCT) b 
where a.SOBHXH=b.SOBHXH""",db_bhxh)

,count(*)
0,1159724


In [18]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH,CMND_ENCRYPT_DECRYPT from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct SOBHXH,CMND_ENCRYPT_DECRYPT from  S_BHXH_ONLINE_NEW_DISTINCT) b 
where a.SOBHXH=b.SOBHXH and a.CMND_ENCRYPT_DECRYPT=b.CMND_ENCRYPT_DECRYPT""",db_bhxh)

,count(*)
0,1152253


In [ ]:
# new + brandname

In [19]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct MABHXH from  BRANDNAME_SI_INDIVIDUAL_DISTINCT) b 
where a.SOBHXH=b.MABHXH""",db_bhxh)

,count(*)
0,1564907


In [ ]:
# new + log

In [20]:
pd.read_sql_query("""select count(*) from (select distinct CMND_ENCRYPT_DECRYPT from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct SO_CMND from  RESPONSE_BHXH_LOG_DISTINCT) b 
where a.CMND_ENCRYPT_DECRYPT=b.SO_CMND""",db_bhxh)

,count(*)
0,286561


In [21]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct MA_SO_MXH from  RESPONSE_BHXH_LOG_DISTINCT) b 
where a.SOBHXH=b.MA_SO_MXH""",db_bhxh)

,count(*)
0,256344


In [22]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH,CMND_ENCRYPT_DECRYPT from  S_BHXH_NEW_DISTINCT) a 
inner join (select distinct MA_SO_MXH,SO_CMND from  RESPONSE_BHXH_LOG_DISTINCT) b 
where a.SOBHXH=b.MA_SO_MXH and a.CMND_ENCRYPT_DECRYPT=b.SO_CMND""",db_bhxh)

,count(*)
0,219605


In [ ]:
# online new + brandname

In [23]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH from  S_BHXH_ONLINE_NEW_DISTINCT) a 
inner join (select distinct MABHXH from  BRANDNAME_SI_INDIVIDUAL_DISTINCT) b 
where a.SOBHXH=b.MABHXH""",db_bhxh)

,count(*)
0,376499


In [ ]:
# online new + log

In [24]:
pd.read_sql_query("""select count(*) from (select distinct CMND_ENCRYPT_DECRYPT from  S_BHXH_ONLINE_NEW_DISTINCT) a 
inner join (select distinct SO_CMND from  RESPONSE_BHXH_LOG_DISTINCT) b 
where a.CMND_ENCRYPT_DECRYPT=b.SO_CMND""",db_bhxh)

,count(*)
0,37827


In [25]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH from  S_BHXH_ONLINE_NEW_DISTINCT) a 
inner join (select distinct MA_SO_MXH from  RESPONSE_BHXH_LOG_DISTINCT) b 
where a.SOBHXH=b.MA_SO_MXH""",db_bhxh)

,count(*)
0,43504


In [26]:
pd.read_sql_query("""select count(*) from (select distinct SOBHXH,CMND_ENCRYPT_DECRYPT from  S_BHXH_ONLINE_NEW_DISTINCT) a 
inner join (select distinct MA_SO_MXH,SO_CMND from  RESPONSE_BHXH_LOG_DISTINCT) b 
where a.SOBHXH=b.MA_SO_MXH and a.CMND_ENCRYPT_DECRYPT=b.SO_CMND""",db_bhxh)

,count(*)
0,35367


In [ ]:
 # brandname + log

In [27]:
pd.read_sql_query("""select count(*) from (select distinct MA_SO_MXH from  RESPONSE_BHXH_LOG_DISTINCT) a 
inner join (select distinct MABHXH from  BRANDNAME_SI_INDIVIDUAL_DISTINCT) b 
where a.MA_SO_MXH=b.MABHXH""",db_bhxh)

,count(*)
0,579509


In [30]:
pd.read_sql_query("""select count(*) from (select distinct CMND_ENCRYPT_DECRYPT,SOBHXH from S_BHXH_NEW_DISTINCT)""",db_bhxh)

,count(*)
0,6417268


In [29]:
pd.read_sql_query("""select count(*) from (select distinct CMND_ENCRYPT_DECRYPT,SOBHXH from S_BHXH_ONLINE_NEW_DISTINCT)""",db_bhxh)

,count(*)
0,1159724


In [32]:
pd.read_sql_query("""select count(*) from (select distinct SO_CMND,MA_SO_MXH from RESPONSE_BHXH_LOG_DISTINCT)""",db_bhxh)

,count(*)
0,1664843
